In [39]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch
from torch import nn

In [53]:
from sklearn.calibration import LabelEncoder


class IrisDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        label_encoder = LabelEncoder()
        self.data['Species'] = label_encoder.fit_transform(self.data['Species'])
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {
            'features': torch.tensor(self.data.iloc[idx, 1:5].values, dtype=torch.float32),
            'label': torch.tensor(self.data.iloc[idx, 5], dtype=torch.long)
        }

        if self.transform:
            sample = self.transform(sample)

        return sample


In [91]:
csv_file = './data/Iris.csv'
iris_dataset = IrisDataset(csv_file)

dataset_size = len(iris_dataset)
train_size = int(0.6 * dataset_size)
test_size = dataset_size - train_size

train_dataset, test_dataset = random_split(iris_dataset, [train_size, test_size])

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [92]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using {device} device")

Using cpu device


In [93]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=3, bias=True)
  )
)


In [94]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [96]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, sample in enumerate(dataloader):
        X, y = sample['features'].to(device), sample['label'].to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch, sample in enumerate(dataloader):
            X, y = sample['features'].to(device), sample['label'].to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [97]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.560161  [   16/   90]
Test Error: 
 Accuracy: 61.7%, Avg loss: 1.093398 

Epoch 2
-------------------------------
loss: 1.101016  [   16/   90]
Test Error: 
 Accuracy: 31.7%, Avg loss: 1.010000 

Epoch 3
-------------------------------
loss: 0.974854  [   16/   90]
Test Error: 
 Accuracy: 35.0%, Avg loss: 0.980239 

Epoch 4
-------------------------------
loss: 0.907147  [   16/   90]
Test Error: 
 Accuracy: 61.7%, Avg loss: 0.981174 

Epoch 5
-------------------------------
loss: 0.907235  [   16/   90]
Test Error: 
 Accuracy: 61.7%, Avg loss: 0.951560 

Epoch 6
-------------------------------
loss: 0.952070  [   16/   90]
Test Error: 
 Accuracy: 61.7%, Avg loss: 0.942523 

Epoch 7
-------------------------------
loss: 0.892347  [   16/   90]
Test Error: 
 Accuracy: 61.7%, Avg loss: 0.903775 

Epoch 8
-------------------------------
loss: 0.871230  [   16/   90]
Test Error: 
 Accuracy: 61.7%, Avg loss: 0.898578 

Epoch 9
----------------

In [90]:
torch.save(model.state_dict(), "./Model/Iris.pth")
print("Saved PyTorch Model State to Iris.pth")

Saved PyTorch Model State to Iris.pth
